# 10-armed Testbed

In [37]:
import numpy as np

class Bandit:
    # region Constructor

    def __init__(self, arms_number: int = 10, use_sample_averages: bool = False, epsilon=0., initial_action_value_estimates=0., confidence_level=None,
                 use_gradient: bool = False, step_size=0.1, use_gradient_baseline: bool = False, true_expected_reward=0.):
        # region Summary
        """
        k-armed Bandit.
        :param arms_number: (denoted as k) number of bandit's arms
        :param use_sample_averages: if True, use sample-average method for estimating action values
        :param epsilon: (denoted as ε) probability for exploration in ε-greedy algorithm
        :param initial_action_value_estimates: (denoted as 𝑄_1(𝑎)) initial estimation for each action value
        :param confidence_level: (denoted as 𝑐) if not None, use Upper-Confidence-Bound (UCB) action selection
        :param use_gradient: if True, use Gradient Bandit Algorithm (GBA)
        :param step_size: (denoted as 𝛼) constant step size for updating estimates
        :param use_gradient_baseline: if True, use average reward as baseline for GBA
        :param true_expected_reward: true expected rewards selected from normal (Gaussian) distribution with μ=4 mean and σ=1 variance
        """
        # endregion Summary

        # region Body

        self.k = arms_number
        self.actions = np.arange(self.k)

        # Value of each action is expected or mean reward given that that action is selected (denoted as 𝑞_∗(𝑎))
        self.action_values = None

        # Estimated value of each action (denoted as 𝑄_𝑡(𝑎))
        self.estimated_action_values = None

        # region Action-Value Methods

        # region Sample-average Method

        self.use_sample_averages = use_sample_averages

        # endregion Sample-average Method

        # region Action Selection Methods

        # region ε-greedy

        self.epsilon = epsilon

        # endregion ε-greedy

        # region Optimistic Initial Values

        self.initial_action_value_estimates = initial_action_value_estimates

        # endregion Optimistic Initial Values

        # region UCB

        self.confidence_level = confidence_level

        # Time steps
        self.time = 0

        # Number of times each action has been selected (denoted as 𝑁_𝑡(𝑎))
        self.action_selection_count = None

        # endregion UCB

        # region GBA

        self.use_gradient = use_gradient

        # Probability of taking action 𝑎 at time 𝑡 (denoted as 𝜋_𝑡(𝑎))
        self.action_probability = None

        self.step_size = step_size

        # Average of the rewards up to (but not including) time 𝑡 (denoted as 𝑅̅_𝑡)
        self.average_reward = 0

        self.use_gradient_baseline = use_gradient_baseline

        self.true_expected_reward = true_expected_reward

        # endregion GBA

        # endregion Action Selection Methods

        # endregion Action-Value Methods

        # Optimal action
        self.optimal_action = None

        # endregion Body

    # endregion Constructor

    # region Functions

    def initialize(self):
        # region Summary
        """
        Initialize action parameters
        """
        # endregion Summary

        # region Body

        # Initialize action values according to a normal (Gaussian) distribution with μ=0 mean and σ=1 variance.
        # In case of GBA, add true_expected_reward != 0.
        self.action_values = np.random.randn(self.k) + self.true_expected_reward

        # In case of realistic initial values, initialize estimated action values with 0s.
        # In case of optimistic initial values, add initial_action_value_estimates != 0
        self.estimated_action_values = np.zeros(self.k) + self.initial_action_value_estimates

        # Set time steps to 0
        self.time = 0

        # Initialize number of times each action has been selected to 0 (none of actions has been selected yet)
        self.action_selection_count = np.zeros(self.k)

        # Optimal action is the action with the highest value
        self.optimal_action = np.argmax(self.action_values)

        # endregion Body

    def act(self):
        # region Summary
        """
        Get an action for this bandit.
        :return: Action
        """
        # endregion Summary

        # region Body

        # region ε-greedy

        # ε-greedy action selection: every once in a while, with small probability ε, select randomly from among all the actions with equal probability, independently of the action-value estimates.
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.actions)

        # endregion ε-greedy

        # region Greedy

        # Greedy action selection: select one of the actions with the highest estimated value, that is, one of the greedy actions.
        # If there is more than one greedy action, then a selection is made among them in some arbitrary way, perhaps randomly.
        action = np.random.choice(np.where(self.estimated_action_values == np.max(self.estimated_action_values))[0])
        return action
        # endregion Greedy

        # endregion Body

    def step(self, action):
        # region Summary
        """
        Update estimated action value and return reward for this action.
        :param action: Action
        :return: Reward
        """
        # endregion Summary

        # region Body

        # When a learning method applied to that bandit problem selected action 𝐴_𝑡 at time step 𝑡, the actual reward, 𝑅_𝑡, was selected from
        # a normal (Gaussian) distribution with μ = 𝑞_∗(𝑎) mean and σ = 1 variance
        actual_reward = np.random.randn() + self.action_values[action]

        # Add 1 to time step
        self.time += 1

        # Add 1 to number of times this action has been selected
        self.action_selection_count[action] += 1

        # The average of the rewards can be computed incrementally
        # The Bandit Gradient Algorithm as Stochastic Gradient Ascent
        self.average_reward += (actual_reward - self.average_reward) / self.time

        if self.use_sample_averages: # Update estimated action values using sample-average method
            # Incremental Implementation (Equation 2.3)
            self.estimated_action_values[action] += (actual_reward - self.estimated_action_values[action]) / self.action_selection_count[action]

        elif self.use_gradient: # Update estimated action values using GBA
            one_hot_encoding = np.zeros(self.k)
            one_hot_encoding[action] = 1

            # The average of the rewards can serve as a baseline with which the reward is compared.
            baseline = self.average_reward if self.use_gradient_baseline else 0

            # A natural learning algorithm for soft-max action preferences based on the idea of stochastic gradient ascent:
            # on each step, after selecting action 𝐴_𝑡 and receiving the reward 𝑅_𝑡, the action preferences are updated by Equation 2.12:
            self.estimated_action_values += self.step_size * (actual_reward - baseline) * (one_hot_encoding - self.action_probability)

        else: # Update estimated action values with constant step size
            # Incremental Implementation (Equation 2.3) with constant step size parameter
            self.estimated_action_values[action] += self.step_size * (actual_reward - self.estimated_action_values[action])

        return actual_reward

        # endregion Body

    # endregion Functions


In [38]:
import numpy as np
from tqdm import trange
import matplotlib
import matplotlib.pyplot as plt
import sys
sys.path.append(r"C:\Users\lilit\PycharmProjects\Reinforcement_Learning\Ten-Armed-Testbed")

# from src.bandit import Bandit


matplotlib.use('Agg')

In [39]:
def simulate(runs, times, bandits):
    # region Summary
    """
    For any learning method, we can measure its performance and behavior as it improves with experience over 1000 time steps 
    when applied to 1 of the bandit problems. This makes up 1 run. Repeating this for 2000 independent runs, each with a different 
    bandit problem, we obtained measures of the learning algorithm’s average behavior.
    :param runs: Number of runs
    :param times: Number of times
    :param bandits: Bandit problems
    :return: Optimal action count mean and reward mean
    """
    # endregion Summary
    
    # region Body
    
    # Prepare a matrix filled with 0s for rewards
    rewards = np.zeros((len(bandits),runs, times))
    
    # Prepare a matrix filled with 0s for optimal action counts that has the same shape as rewards matrix
    optimal_action_counts = np.zeros(rewards.shape)

    # For every bandit
    for i, bandit in enumerate(bandits):
        # for every run
        for run in trange(runs):
            # initialize bandit
            bandit.initialize()
            
            # for every time step
            for time in range(times):
                # select an action
                action = bandit.act()
                
                # get the reward
                rewards[i, run, time] = bandit.step(action)
                
                # if the selected action is optimal for bandit
                if action == bandit.optimal_action:
                    # change the corresponding 0 in the optimal action counts matrix to 1
                    optimal_action_counts[i, run, time] = 1

    return optimal_action_counts.mean(axis=1), rewards.mean(axis=1)

    # endregion Body

## 1. Reward Distribution

In [40]:
# Plot an example reward distribution
plt.violinplot(dataset=np.random.randn(200, 10) + np.random.randn(10))
plt.title("Figure 2.1")
plt.xlabel("Action")
plt.ylabel("Reward distribution")
plt.savefig("../generated_images/figure_2_1.png")
plt.close()

## 2. Greedy Action Selection VS ε-greedy Action Selection

In [41]:
# Create a list of epsilons with 0, 0.1 and 0.01 values
epsilons = [0, 0.1, 0.01]

# Create a list of bandits (1 bandit for every epsilon) where every bandit uses sample-average method
bandits = [Bandit(epsilon=epsilon, use_sample_averages=True) for epsilon in epsilons]

In [42]:
# Define number of runs
runs = 2000

# Define number of times
time = 1000

# Simulate optimal action counts and rewards
optimal_action_counts, rewards = simulate(runs, time, bandits)

100%|██████████| 2000/2000 [00:18<00:00, 108.79it/s]


In [43]:
# Plotting
plt.figure(figsize=(10, 20))

<Figure size 1000x2000 with 0 Axes>

In [44]:
plt.subplot(2, 1, 1)
for epsilon, rewards in zip(epsilons, rewards):
    plt.plot(rewards, label="$\epsilon = %.02f$" % epsilon)
plt.title("Figure 2.2")
plt.xlabel("Steps")
plt.ylabel("Average reward")
plt.legend()

In [45]:
plt.subplot(2, 1, 2)
for epsilon, counts in zip(epsilons, optimal_action_counts):
    plt.plot(counts, label="$\epsilon = %.02f$" % epsilon)
plt.xlabel("Steps")
plt.ylabel("% Optimal action")
plt.legend()

In [46]:
plt.savefig("../generated_images/figure_2_2.png")
plt.close()

## 3. Optimistic Initial Values VS Realistic Initial Values

In [ ]:
# Create a list of 2 bandits where:
# 1. 1st bandit: ε = 0, 𝑄_1(𝑎) = 5, 𝛼 = 0.1,
# 2. 2nd bandit: ε = 0.1, 𝑄_1(𝑎) = 0, 𝛼 = 0.1


In [ ]:
# Define number of runs


# Define number of times


# Simulate optimal action counts


In [ ]:
# Plotting


## 4. Upper-Confidence-Bound (UCB) Action Selection

In [ ]:
# Create a list of 2 bandits where:
# 1. 1st bandit: ε = 0, 𝑐 = 2, uses sample-average method,
# 2. 2nd bandit: ε = 0.1, uses sample-average method


In [ ]:
# Define number of runs


# Define number of times


# Simulate average rewards


In [ ]:
# Plotting


## 5. Gradient Bandit Algorithms (GBA)

In [ ]:
# Create a list of 4 bandits where:
# 1. 1st bandit: uses GBA, 𝛼 = 0.1, uses average reward as baseline for GBA, expects true reward of 4,
# 2. 2nd bandit: uses GBA, 𝛼 = 0.1, doesn't use average reward as baseline for GBA, expects true reward of 4,
# 3. 3rd bandit: uses GBA, 𝛼 = 0.4, uses average reward as baseline for GBA, expects true reward of 4,
# 4. 4th bandit: uses GBA, 𝛼 = 0.4, doesn't use average reward as baseline for GBA, expects true reward of 4


In [ ]:
# Define number of runs


# Define number of times


# Simulate optimal action counts\


In [ ]:
# Labels


In [ ]:
# Plotting
